In [1]:
import tensorflow as tf
import time
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.utils import to_categorical

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train, 10), to_categorical(y_test, 10)

In [5]:
def create_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(256, activation='relu'),
        Dense(256, activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

results = {}

In [7]:
with tf.device('/CPU:0'):
    model_cpu = create_model()
    start_time = time.time()
    model_cpu.fit(x_train, y_train, epochs=10, batch_size=128, verbose=0)
    cpu_time = time.time() - start_time
    _, cpu_accuracy = model_cpu.evaluate(x_test, y_test, verbose=0)
    results['CPU'] = {'Time (s)': cpu_time, 'Accuracy': cpu_accuracy}

C:\Users\indum\anaconda3\Lib\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
if tf.config.list_physical_devices('GPU'):
    model_gpu = create_model()
    start_time = time.time()
    model_gpu.fit(x_train, y_train, epochs=10, batch_size=128, verbose=0)
    gpu_time = time.time() - start_time
    _, gpu_accuracy = model_gpu.evaluate(x_test, y_test, verbose=0)
    results['GPU'] = {'Time (s)': gpu_time, 'Accuracy': gpu_accuracy}
else:
    results['GPU'] = {'Time (s)': 'N/A', 'Accuracy': 'N/A'}

In [11]:
print("\n🔹 Benchmark Results:")
print("{:<10} {:<15} {:<10}".format('Device', 'Time (s)', 'Accuracy'))
for device, metrics in results.items():
    print("{:<10} {:<15} {:<10}".format(device, round(metrics['Time (s)'], 2) if metrics['Time (s)'] != 'N/A' else 'N/A',
                                        round(metrics['Accuracy'], 4) if metrics['Accuracy'] != 'N/A' else 'N/A'))

print("\nNote: GPU training is generally faster due to parallel computations, especially beneficial for deeper networks.")


🔹 Benchmark Results:
Device     Time (s)        Accuracy  
CPU        44.86           0.9789    
GPU        N/A             N/A       

Note: GPU training is generally faster due to parallel computations, especially beneficial for deeper networks.
